In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime, random
import matplotlib.dates as mdates

def MakeDataFromText(filepath):
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif len(line)==8:               # If it is a time stamp
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h) - 3, int(m), int(s))) # Minus 3 for Pacific Time
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db

class Person:
    """
    Class to represent each individal poutfit being recorded during Deployment, made up of garments.
    """
    def __init__(
        self, Name:str, color:str,
        backneck:str,   back:str,        chest:str, 
        leftab:str,     rightab:str, 
        rightuparm:str, rightlowarm:str, 
        leftuparm:str,  leftlowarm:str, 
        beanie1:str,    beanie2:str,     beanie3:str, beanieR:str, beanieL:str
    ):
        self.Backneck    = backneck        # Tag ID's for the shirt
        self.Back        = back
        self.Chest       = chest
        self.Leftab      = leftab
        self.Rightab     = rightab
        self.Rightuparm  = rightuparm
        self.Rightlowarm = rightlowarm
        self.Leftuparm   = leftuparm
        self.Leftlowarm  = leftlowarm
        self.Beanie1     = beanie1         # Tag ID's for the beanies
        self.Beanie2     = beanie2
        self.Beanie3     = beanie3
        self.BeanieR     = beanieR
        self.BeanieL     = beanieL

        self.Name = Name                   # Soldier's Name

        self.ShirtMapping = {
            self.Backneck:    Name + " Back Neck",
            self.Back:        Name + " Back",
            self.Chest:       Name + " Chest",
            self.Leftab:      Name + " Left Abdomen",
            self.Rightab:     Name + " Right Abdomen",
            self.Rightuparm:  Name + " Right Upper Arm",
            self.Rightlowarm: Name + " Right Lower Arm",
            self.Leftlowarm:  Name + " Left Lower Arm",
            self.Leftuparm:   Name + " Left Upper Arm",
        }

        self.TagList = [
            self.Backneck,   self.Back, self.Chest,
            self.Leftab,     self.Rightab,
            self.Rightuparm, self.Rightlowarm,
            self.Leftuparm,  self.Leftlowarm,
            self.Beanie1,    self.Beanie2, self.Beanie3, self.BeanieR, self.BeanieL
        ]

        self.BeanieList = [self.Beanie1, self.Beanie2, self.Beanie3, self.BeanieR, self.BeanieL]
        self.Color = color

In [ ]:
person6  = Person("Pa. P.", "#de0413", "71BB", "7705", "B25E", "3247", "A9B5", "6C38", "7662", "A983", "098F", "B644", "7BCC", "B576", "70D1", "4D84");
person14 = Person("Jo.",    "#9e05a1", "462C", "5B60", "7415", "6310", "1851", "616A", "5DAA", "6D28", "9991", "3B0E", "5374", "682C", "0A50", "1543");
person28 = Person("Ga. C.", "#0909eb", "8977", "4EB3", "AA5E", "6626", "9ABE", "6CD0", "AB8E", "14AA", "6E94", "895D", "571F", "2D68", "8121", "198C");
person2  = Person("De.",    "#078c26", "333B", "289B", "2473", "231D", "9879", "4067", "5FB6", "169E", "8D50", "74C4", "73DC", "DC4A", "884D", "1BA4");
person17 = Person("Br. L.", "#e8c100", "6CC7", "8FB7", "799D", "6F8D", "6332", "5A1E", "92A5", "4A3D", "3EC9", "16AB", "A66C", "6162", "A686", "7E1B");
person15 = Person("Ja.",    "#0c008f", "9C54", "9968", "65E1", "5E23", "ACB3", "7AD3", "99A1", "B19D", "43C6", "7DCB", "9D35", "6FD0", "34CE", "62E3");
person16 = Person("Ma. C.", "#02591e", "959E", "1C5F", "5A59", "077A", "902A", "3B60", "8199", "4469", "813A", "B7A4", "5163", "4DA5", "8995", "7972");
person13 = Person("Wi. D.", "#a60000", "A033", "A0C0", "892F", "627F", "6BD9", "3DB0", "8C97", "4ECF", "A73C", "90AE", "78D3", "82D8", "312D", "2A29");
person32 = Person("Se.",    "#7d0080", "2990", "1EC3", "51E2", "3783", "AE5E", "4575", "0688", "A0B0", "1BA6", "3C3E", "1989", "7C2C", "42D7", "6129");
person19 = Person("Ha.",    "#003952", "697F", "78A5", "5D0E", "7EC6", "AE59", "8158", "4A9B", "1D44", "2122", "5C02", "2D90", "6DAB", "3374", "B1AB");
person26 = Person("Ka.",    "#965500", "4813", "2791", "1F7A", "6F4A", "2383", "2F66", "2E5C", "926A", "2E56", "158F", "5B06", "B28A", "73C6", "819C");

In [ ]:
b_gym1 = MakeDataFromText("MWTC/baseline_gym_1.txt")
b_gym2 = MakeDataFromText("MWTC/baseline_gym_2.txt")
b_gym3 = MakeDataFromText("MWTC/baseline_gym_3.txt")
b_gym4 = MakeDataFromText("MWTC/baseline_gym_4.txt")

combined = dict()

for d in [b_gym1, b_gym2, b_gym3, b_gym4]:
    for k,v in d.items():
        if k in combined: # if tag is already in dict
            for i in range(len(v[0])):
                if v[0][i] in combined[k]:
                    combined[k][v[0][i]].append(v[1][i])
                else:
                    combined[k][v[0][i]] = [v[1][i]]
        else:             # if tag is not in dict
            combined[k] = dict()
            for i in range(len(v[0])):
                combined[k][v[0][i]] = [v[1][i]]

for k,v in combined.items():
    for k2,v2 in v.items():
        combined[k][k2] = sum(v2)/len(v2)

In [ ]:
import matplotlib.dates as mdates
myFmt = mdates.DateFormatter('%H:%M:%S')
fig, ax = plt.subplots(figsize=(13, 8))
ax.xaxis.set_major_formatter(myFmt)

dates = [
    [datetime.datetime(1970, 2, 1, 7, 39, 0), datetime.datetime(1970, 2, 1, 14, 2, 0)],
    [datetime.datetime(1970, 2, 1, 14, 2, 0), datetime.datetime(1970, 2, 1, 14, 15, 0)],
    [datetime.datetime(1970, 2, 1, 14, 15, 0), datetime.datetime(1970, 2, 1, 14, 20, 0)],
    [datetime.datetime(1970, 2, 1, 14, 20, 0), datetime.datetime(1970, 2, 1, 14, 30, 0)],
    [datetime.datetime(1970, 2, 1, 14, 22, 0), datetime.datetime(1970, 2, 1, 14, 37, 0)],
    [datetime.datetime(1970, 2, 1, 14, 30, 0), datetime.datetime(1970, 2, 1, 14, 45, 0)],
    [datetime.datetime(1970, 2, 1, 14, 43, 0), datetime.datetime(1970, 2, 1, 15, 0, 0)],
    [datetime.datetime(1970, 2, 1, 14, 48, 0), datetime.datetime(1970, 2, 1, 15, 3, 0)],
    [datetime.datetime(1970, 2, 1, 15, 0, 0), datetime.datetime(1970, 2, 1, 15, 13, 0)],
    [datetime.datetime(1970, 2, 1, 15, 11, 0), datetime.datetime(1970, 2, 1, 15, 18, 0)],
    [datetime.datetime(1970, 2, 1, 15, 17, 0), datetime.datetime(1970, 2, 1, 15, 27, 0)],
]

people = [person26, person17, person15, person32, person19, person13, person28, person2, person14, person6, person16]

for k,v in combined.items():
    id = k[-4:]

    for p in range(11):
        if id in people[p].TagList:
            s = sorted(v.items())
            x = [i[0] for i in s if i[0] > dates[p][0] and i[0] < dates[p][1]]
            y = [i[1] for i in s if i[0] > dates[p][0] and i[0] < dates[p][1]]
            plt.plot(x, y, label=people[p].Name, color=people[p].Color, marker=".")


ax.set_ylabel("Temperature (°C)", fontsize=15)
ax.set_xlabel("Time (H:M:S), Pacific Time", fontsize=15)
ax.set_title("Baseline Test in Gymnasium", fontsize=15)
ax.set_ylim(15.0, 40.0)

# Legend Code
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), loc="lower left")